# Librerias

In [8]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import plotly.graph_objs as go
from collections import defaultdict
import statistics
import json
import re
import matplotlib.pyplot as plt
from datetime import timedelta
import plotly.express as px
from sklearn.preprocessing import OneHotEncoder
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import xml.etree.ElementTree as ET


# Codigo de main

In [9]:
import re
import datetime, time

#Esta clase sirve para parsear timestamps, recibe 2 y hace la diferencia
class Tiempo:
    def __init__ (self, inicio, final = -1, seconds = -1): #Necesario para tener 2 constructores
        self.positivo = True;
        if final == -1: #Primer constructor (1 argumento)

            self.hours = 0
            self.minutes = 0
            self.seconds = 0

            erHours = re.compile(r'\d*h')
            erMinutes = re.compile(r'\d*m')
            erSeconds = re.compile(r'\d*s')
            h = erHours.search(inicio)
            m = erMinutes.search(inicio)
            s = erSeconds.search(inicio)

            if h != None:
                self.hours = h[0][:-1]
            if m != None:
                self.minutes = m[0][:-1]
            if s != None:
                self.seconds = s[0][:-1]
                
            self.hours = int(self.hours)
            self.minutes = int(self.minutes)
            self.seconds = int(self.seconds)
                
            if self.seconds >= 60:
                self.minutes += int(self.seconds/60)
                self.seconds = self.seconds%60
            if self.minutes >= 60:
                self.hours += int(self.minutes/60)
                self.minutes = self.minutes%60
            
        elif seconds == -1: #Segundo constructor (2 aregumentos)
            self.hours = 0
            self.minutes = 0
            self.seconds = 0

            fin = time.mktime(datetime.datetime.strptime(final, "%Y-%m-%dT%H:%M:%S.%fZ").timetuple())
            ini = time.mktime(datetime.datetime.strptime(inicio, "%Y-%m-%dT%H:%M:%S.%fZ").timetuple())

            hoursRaw = int((fin - ini)/3600)
            minutesRaw = int((fin - ini)/60)
            secondsRaw = int(fin - ini)

            if hoursRaw != 0:
                self.hours = hoursRaw
                minutesRaw = int(((fin - ini)%3600)/60)
            if minutesRaw != 0:
                self.minutes = minutesRaw
                secondsRaw = int((fin - ini)%60)
            self.seconds = secondsRaw
            
        else: #Tercer constructor (3 argumentos)
            self.hours = inicio
            self.minutes = final
            self.seconds = seconds
            
        self.hours = int(self.hours)
        self.minutes = int(self.minutes)
        self.seconds = int(self.seconds)
            
    def toString(self):
        return self.__repr__()
    
    def __int__ (self):
        return self.hours*3600 + self.minutes*60 + self.seconds

    def __repr__(self):
        t=""
        if self.hours !=0:
            t += str(self.hours) + "h/"
        if self.minutes !=0:
            t += str(self.minutes) + "m/"
        if self.seconds !=0:
            t += str(self.seconds) + "s"
        else:
            t = t[:-1]
        if self.hours == 0 and self.minutes == 0 and self.seconds == 0:
            return "0s"
        return t
    
    def __add__(self, t): #Sobrecarga del operador +
        s = self.seconds + t.seconds
        m = int(self.minutes) + t.minutes
        h = self.hours + t.hours
        
        if s >= 60:
            s -= 60
            m += 1
        if m >= 60:
            m -= 60
            h += 1
            
        return Tiempo(h,m,s)
        
    
    def __sub__(self, t):
        sSelf = self.hours*3600 + self.minutes*60 + self.seconds
        sT = t.hours*3600 + t.minutes*60 + t.seconds
        
        return Tiempo(str(sSelf-sT) + "s")
    
    def __mul__(self, n): #Sobrecarga del operador *
        s = self.seconds * n
        m = self.minutes * n
        h = self.hours * n
        
        m += (h - int(h))*60
        s += (m - int(m))*60
        
        if s >= 60:
            m += int(s/60)
            s = s%60
        if m >= 60:
            h += int(m/60)
            m = m%60
        
        return Tiempo(h,m,s)
    
    def __truediv__(self, n): #Sobrecarga del operador /
        selfS = self.seconds
        selfM = self.minutes
        selfH = self.hours
        
        h = int(selfH/n)
        selfM += (selfH%n)*60
        m = int(selfM/n)
        selfS += (selfM%n)*60
        s = int(selfS/n)
        
        if s >= 60:
            m += int(s/60)
            s = s%60
        if m >= 60:
            h += int(m/60)
            m = m%60
        return Tiempo(int(h),int(m),int(s))
    
    def __abs__(self):
        return Tiempo(abs(self.hours), abs(self.minutes), abs(self.seconds))
    
    def __le__(self, t):   #Sobrecarga del operado <=  
        sSelf = self.hours*3600 + self.minutes*60 + self.seconds
        if type(t) == Tiempo:
            sT = t.hours*3600 + t.minutes*60 + t.seconds
        else:
            sT = t       
        return sSelf <= sT
    
    def __ge__(self, t):  #Sobrecarga del operado >=
        sSelf = self.hours*3600 + self.minutes*60 + self.seconds
        if type(t) == Tiempo:
            sT = t.hours*3600 + t.minutes*60 + t.seconds
        else:
            sT = t      
        return sSelf >= sT
    
    def __lt__(self, t):     #Sobrecarga del operado <
        sSelf = self.hours*3600 + self.minutes*60 + self.seconds
        if type(t) == Tiempo:
            sT = t.hours*3600 + t.minutes*60 + t.seconds
        else:
            sT = t        
        return sSelf < sT
    
    def __gt__(self, t):   #Sobrecarga del operado >
        sSelf = self.hours*3600 + self.minutes*60 + self.seconds
        if type(t) == Tiempo:
            sT = t.hours*3600 + t.minutes*60 + t.seconds
        else:
            sT = t
        return sSelf > sT
    
    def total_seconds(self):
        return self.hours * 3600 + self.minutes * 60 + self.seconds


In [10]:
#TAMBIEN DEVUELVE LOS INTENTOS INDIVIDUALES DE CADA JUGADOR PARA COMPLETAR CADA NIVEL
def getIntentosMedios_HastaCompletarNivel(intentosNecesarios, intentosOrdenados = False):
    intentosMedios = defaultdict(list)
    intentosCompletarNivelIndividual = defaultdict(defaultdict)
    
    for name in intentosNecesarios:
        for level in intentosNecesarios[name]:
            cont = 0
            for i in intentosNecesarios[name][level]:
                cont += i["intentos"]
                if i["success"] == True:
                    intentosMedios[level].append(cont)
                    intentosCompletarNivelIndividual[name][level] = cont
                    break
            
    for level in intentosMedios:
        intentosMedios[level] = round(statistics.mean(intentosMedios[level]), 2)
    
    if intentosOrdenados:
        intentosMedios = sorted(intentosMedios.items(), key=lambda x: x[1])
    else:
        intentosMedios = list(intentosMedios.items())
    return {"intentosMedios" : intentosMedios, "intentosIndividual" : intentosCompletarNivelIndividual}

def extraerTiemposPorNivelJugador(rawData):
    
    tiempos = defaultdict(defaultdict)
    intentosNecesarios = defaultdict(defaultdict)
    inicioYFinJuego = defaultdict()
    errores = defaultdict(list)
    codJugadores = defaultdict()
    codUltNivel = defaultdict()

    error1 = "No necesario crear variable"
    error2 = "Asignacion mismo valor en variable"
    error3 = "Entero o texto utilizado 2 veces"

    nivelesError1Excepcion = ['variables_2', 'variables_3', 'variables_4', 'variables_6', 'variables_8', 'variables_10', 'types_2', 'basic_operators_1', 'basic_operators_3', 'basic_operators_6']
    nivelesError3Excepcion = ['tutorials_1', 'tutorials_2', 'tutorials_3', 'tutorials_4', 'tutorials_5', 'tutorials_6', 'tutorials_7', 'tutorials_8', 'tutorials_9']

    erLevel = re.compile(r'\blevel$\b')
    erIdLevel = re.compile(r'/')
    
    erInitialized = re.compile(r'\binitialized$\b')
    erCompleted = re.compile(r'\bcompleted$\b')
    erAccessed = re.compile(r'\baccessed$\b')
    erProgressed = re.compile(r'\bprogressed$\b')

    erSeriousGame = re.compile(r'\bserious-game$\b')  
    erCategoryMain = re.compile(r'\bcategories_main$\b')  
    
    for evento in rawData:
        verb = evento["verb"]["id"]
        obj = evento["object"]["definition"]["type"]
        name = evento["actor"]["name"]
        timestamp = evento["timestamp"]
        objectId = evento["object"]["id"]
       
        if erLevel.search(obj): #Si el objeto de la acción es un nivel
            levelCode = erIdLevel.split(objectId)[-1]
            if levelCode != "editor_level":
                if erInitialized.search(verb): #Si la acción es inicio o reinicio
                    if "result" in evento: #Significa que ha iniciado el nivel desde el menu
                        if levelCode in tiempos[name]:
                            intentosNecesarios[name][levelCode].append({"intentos" : 1, "success" : False})
                            tiempos[name][levelCode].append({"ini" : timestamp, "fin" : None, "stars" : ""})
                        else:
                            intentosNecesarios[name][levelCode] = [{"intentos" : 1, "success" : False}]
                            tiempos[name][levelCode] = [{"ini" : timestamp, "fin" : None, "stars" : ""}]
                    else:
                        intentosNecesarios[name][levelCode][-1]["intentos"] += 1

                elif erProgressed.search(verb):
                    cod = ET.fromstring(evento['result']['extensions']['code'])
                    codJugadores[name] = cod
                    codUltNivel[name] = levelCode

                elif erCompleted.search(verb):
                    if evento["result"]["score"]["raw"] > 0 :
                        if levelCode in tiempos[name]:
                            intentosNecesarios[name][levelCode][-1]["success"] = True
                            tiempos[name][levelCode][-1]["fin"] = timestamp
                            tiempos[name][levelCode][-1]["stars"] = evento["result"]["score"]["raw"]

                        if codUltNivel[name] == levelCode: 
                            try:
                                cod = codJugadores[name]
                            except:
                                print(evento)
                                print(timestamp)
                                return
                            vars = defaultdict()
                            values = []

                            for v in cod.find('variables'):
                                vars[v.text] = {"valores" : [], "usado" : 0}

                            for b in cod.find("block[@type='start_start']").iter('block'):
                                tarjeta = b.attrib['type']

                                if tarjeta == "text" or tarjeta == "math_number":
                                    valor = b.find('field').text
                                    if valor in values and levelCode not in nivelesError3Excepcion:
                                        errores[name].append({'error' : error3, 'level' : levelCode})
                                    else:
                                        values.append(valor)

                                elif tarjeta == "variables_set":
                                    varName = b.find('field').text
                                    try:
                                        varValue = b.find('value').find('block').find('field').text
                                    except:
                                        #print(ET.tostring(cod, encoding='utf8', method='xml'))
                                        #print(ET.tostring(b, encoding='utf8', method='xml'))
                                        None                                

                                    if vars[varName]['valores'] and vars[varName]['usado'] < 2 and levelCode not in nivelesError1Excepcion:
                                        errores[name].append({'error' : error1 + " " + varName, 'level' : levelCode})

                                    if varValue in vars[varName]['valores']:
                                        errores[name].append({'error' : error2 + " " + varName, 'level' : levelCode})
                                    else:
                                        vars[varName]['valores'].append(varValue)

                                    vars[varName]['usado'] = 0

                                elif tarjeta == "variables_get":
                                    varName = b.find('field').text
                                    vars[varName]["usado"] += 1

                            for v in vars:
                                if vars[v]['usado'] < 2 and levelCode not in nivelesError1Excepcion:
                                    errores[name].append({'error' : error1 + " " + v, 'level' : levelCode})

                    elif evento["result"]["score"]["raw"] == -1:
                        if levelCode in tiempos[name]:
                            tiempos[name][levelCode][-1]["fin"] = timestamp
                            tiempos[name][levelCode][-1]["stars"] = evento["result"]["score"]["raw"]
        
        elif erSeriousGame.search(obj) and erInitialized.search(verb):
            if name in inicioYFinJuego:
                inicioYFinJuego[name].append({"inicio" : timestamp, "fin" : None})
            else:
                inicioYFinJuego[name] = [{"inicio" : timestamp, "fin" : None}]
        
        if not(erAccessed.search(verb) and erCategoryMain.search(objectId)):
            inicioYFinJuego[name][-1]["fin"] = timestamp
    
    return {"tiempos" : tiempos, "intentosNecesarios" : intentosNecesarios, "inicioYFinJuego" : inicioYFinJuego, "erroresCodigo" : errores}

JSONFile = open('./data/trazasOrdenadas.json')
rawData = json.load(JSONFile)
JSONFile.close()

intentosOrdenados = False
resultados_Tiempos_Nivel_Jugador = extraerTiemposPorNivelJugador(rawData)
intentosMedios_Individual = getIntentosMedios_HastaCompletarNivel(resultados_Tiempos_Nivel_Jugador["intentosNecesarios"], intentosOrdenados)

#pd.DataFrame(intentosMedios_Individual["intentosMedios"])
#pd.DataFrame(intentosMedios_Individual["intentosIndividual"])

In [11]:
#Si tenerEnCuentaMove = False -> cuando se realiza una accion de tipo "move" no se tiene en cuenta para calcular el tiempo entre interacciones, como si no existiese la traza
def extraerTiemposEntreInteracciones2(rawData, tenerEnCuentaMove = False):
    ultimaInteraccionJugador = defaultdict()
    
    tiempoInicio_Interaccion_Jugador = defaultdict(defaultdict) #Tiempo entre inicio de nivel y la siguiente interaccion (reinicio, completado o interaccion con tarjeta)
    tiempoInteraccion_Completado_Jugador = defaultdict(defaultdict) #Tiempo entre una interaccion (inicio, reinicio o interaccion con tarjeta) y terminar el nivel
    tiempoInteraccion_Interaccion_Jugador = defaultdict(defaultdict) #Tiempo entre una interaccion con tarjeta y otra interaccion con tarjeta
    
    anomalias = []
    
    erInteracted = re.compile(r'\binteracted$\b')
    erGameObject = re.compile(r'\bgame-object$\b')
    erLevelExitButton = re.compile(r'\blevel_exit_button$\b') 
    erInitialized = re.compile(r'\binitialized$\b')
    erSeriousGame = re.compile(r'\bserious-game$\b')    
    erCompleted = re.compile(r'\bcompleted$\b')
    erLevel = re.compile(r'\blevel$\b')

        
    for e in rawData:
        verb = e["verb"]["id"]
        obj = e["object"]["definition"]["type"]
        objectId = e["object"]["id"]
        name = e["actor"]["name"]
        timestamp = e["timestamp"]
        
        erIdLevel = re.compile(r'/')
        levelCode = erIdLevel.split(objectId)[-1]
        
        if(levelCode != "editor_level"):
            
            if erSeriousGame.search(obj) and erInitialized.search(verb): #Si inicia el juego se reinicia su ultima interaccion
                if name in ultimaInteraccionJugador:
                    del ultimaInteraccionJugador[name]

            elif erLevel.search(obj):
                if erInitialized.search(verb): #Significa que ha iniciado o reiniciado el nivel
                    if "result" in e: #Inicia el nivel desde el menu
                        ultimaInteraccionJugador[name] = {"t" : timestamp, "accion" : "inicio"}
                    else: #Reinicia el nivel
                        try:
                            t = Tiempo(ultimaInteraccionJugador[name]["t"], timestamp)
                            
                            if ultimaInteraccionJugador[name]["accion"] == "inicio":
                                #Añadimos la dif de tiempo al diccionario del jugador
                                if levelCode in tiempoInicio_Interaccion_Jugador[name]:
                                    tiempoInicio_Interaccion_Jugador[name][levelCode].append({"tiempo" : t, "accion" : "reinicio"})
                                else:
                                    tiempoInicio_Interaccion_Jugador[name][levelCode] = [{"tiempo" : t, "accion" : "reinicio"}]
                            #Actualizamos ultima interaccion
                            ultimaInteraccionJugador[name] = {"t" : timestamp, "accion" : "reinicio"}
                        except:
                            #Si entra aqui es porque ha reiniciado un nivel que no habia iniciado
                            #Trazas estan mal
                            anomalias.append({"Jugador" : name, "Timestamp" : timestamp, "Nivel" : levelCode, "Motivo" : "Se ha reiniciado un nivel que no estaba iniciado"})

                elif erCompleted.search(verb) and e["result"]["score"]["raw"] != 0:
                    #Si completa un nivel o sale al menu se reinicia su ultima interaccion, si falla el intento pero vuelve a intentarlo no se reinicia
                    #Si raw == 0 significa que se reinicia el nivel
                    try:
                        t = Tiempo(ultimaInteraccionJugador[name]["t"], timestamp)                        
                        #Añadimos la dif de tiempo al diccionario del jugador
                        if ultimaInteraccionJugador[name]["accion"] == "inicio":
                                #Añadimos la dif de tiempo al diccionario del jugador
                            if levelCode in tiempoInicio_Interaccion_Jugador[name]:
                                tiempoInicio_Interaccion_Jugador[name][levelCode].append({"tiempo" : t, "accion" : "terminado", "exito" : e["result"]["success"]})
                            else:
                                tiempoInicio_Interaccion_Jugador[name][levelCode] = [{"tiempo" : t, "accion" : "terminado", "exito" : e["result"]["success"]}]
                                
                            if levelCode in tiempoInteraccion_Completado_Jugador[name]:
                                tiempoInteraccion_Completado_Jugador[name][levelCode].append({"tiempo" : t, "accion_anterior" : "inicio"})
                            else:
                                tiempoInteraccion_Completado_Jugador[name][levelCode] = [{"tiempo" : t, "accion_anterior" : "inicio"}]
                                    
                        elif ultimaInteraccionJugador[name]["accion"] == "reinicio":
                            if levelCode in tiempoInteraccion_Completado_Jugador[name]:
                                tiempoInteraccion_Completado_Jugador[name][levelCode].append({"tiempo" : t, "accion_anterior" : "reinicio"})
                            else:
                                tiempoInteraccion_Completado_Jugador[name][levelCode] = [{"tiempo" : t, "accion_anterior" : "reinicio"}]

                        elif ultimaInteraccionJugador[name]["accion"] == "interaccion":
                            if levelCode in tiempoInteraccion_Completado_Jugador[name]:
                                tiempoInteraccion_Completado_Jugador[name][levelCode].append({"tiempo" : t, "accion_anterior" : "interaccionTarjeta"})
                            else:
                                tiempoInteraccion_Completado_Jugador[name][levelCode] = [{"tiempo" : t, "accion_anterior" : "interaccionTarjeta"}]
                        #Actualizamos ultima interaccion, la borramos
                        del ultimaInteraccionJugador[name]
                    except:
                        #Si entra aqui es porque se ha completado un nivel que no se habia iniciado
                        #Probablemente por anomalias en las trazas
                        anomalias.append({"Jugador" : name, "Timestamp" : timestamp, "Nivel" : levelCode, "Motivo": "Se ha completado un nivel que no estaba iniciado"})

            elif erGameObject.search(obj) and erInteracted.search(verb):
                if "result" in e and "extensions" in e["result"] and "level" in e["result"]["extensions"]:
                    levelCode = e["result"]["extensions"]["level"]
                    if not erLevelExitButton.search(objectId) and levelCode != "editor_level":
                        if name in ultimaInteraccionJugador:
                            if e["result"]["extensions"]["action"] != "move" or tenerEnCuentaMove: #<-- Que accion realiza
                                t = Tiempo(ultimaInteraccionJugador[name]["t"], timestamp)
                                #Añadimos la dif de tiempo al diccionario del jugador

                                if ultimaInteraccionJugador[name]["accion"] == "inicio":
                                    if levelCode in tiempoInicio_Interaccion_Jugador[name]:
                                        tiempoInicio_Interaccion_Jugador[name][levelCode].append({"tiempo" : t, "accion" : "interaccionTarjeta"})
                                    else:
                                        tiempoInicio_Interaccion_Jugador[name][levelCode] = [{"tiempo" : t, "accion" : "interaccionTarjeta"}]

                                if ultimaInteraccionJugador[name]["accion"] == "interaccionTarjeta":
                                    if levelCode in tiempoInteraccion_Interaccion_Jugador[name]:
                                        tiempoInteraccion_Interaccion_Jugador[name][levelCode].append({"tiempo" : t, "accion_anterior" : "interaccionTarjeta"})
                                    else:
                                        tiempoInteraccion_Interaccion_Jugador[name][levelCode] = [{"tiempo" : t, "accion_anterior" : "interaccionTarjeta"}]

                                ultimaInteraccionJugador[name] = {"t" : timestamp, "accion" : "interaccionTarjeta"}
                        else:
                            anomalias.append({"Jugador" : name, "Timestamp" : timestamp, "Nivel" : levelCode, "Motivo" : "Se ha interactuado en un nivel que no estaba iniciado"})
                        
                    
    # Dataframe 1: tiempoInteraccion_Completado_Jugador
    df_tiempoInteraccion_Completado_Jugador = pd.DataFrame(tiempoInteraccion_Completado_Jugador)
    df_tiempoInteraccion_Completado_Jugador = df_tiempoInteraccion_Completado_Jugador.applymap(lambda x: np.nan if not isinstance(x, list) or len(x) == 0 else int(x[-1]["tiempo"]))
    df_tiempoInteraccion_Completado_Jugador.index.name = "nivel"

    # Dataframe 2: tiempoInicio_Interaccion_Jugador
    df_tiempoInicio_Interaccion_Jugador = pd.DataFrame(tiempoInicio_Interaccion_Jugador)
    df_tiempoInicio_Interaccion_Jugador = df_tiempoInicio_Interaccion_Jugador.applymap(lambda x: np.nan if not isinstance(x, list) or len(x) == 0 else int(x[0]["tiempo"]))
    df_tiempoInicio_Interaccion_Jugador.index.name = "nivel"

    # Dataframe 3: tiempoInteraccion_Interaccion_Jugador
    df_tiempoInteraccion_Interaccion_Jugador = pd.DataFrame(tiempoInteraccion_Interaccion_Jugador)
    df_tiempoInteraccion_Interaccion_Jugador = df_tiempoInteraccion_Interaccion_Jugador.applymap(lambda x: np.nan if not isinstance(x, list) or len(x) == 0 else sum(int(tiempo["tiempo"]) for tiempo in x) / len(x))
    df_tiempoInteraccion_Interaccion_Jugador.index.name = "nivel"


    return df_tiempoInteraccion_Completado_Jugador, df_tiempoInicio_Interaccion_Jugador, df_tiempoInteraccion_Interaccion_Jugador

df_tiempoInteraccion_Completado_Jugador, df_tiempoInicio_Interaccion_Jugador, df_tiempoInteraccion_Interaccion_Jugador = extraerTiemposEntreInteracciones2(rawData, True)
#df_tiempoInteraccion_Completado_Jugador
#df_tiempoInicio_Interaccion_Jugador
#df_tiempoInteraccion_Interaccion_Jugador


In [12]:
resultados_Tiempos_Nivel_Jugador = extraerTiemposPorNivelJugador(rawData)
res = defaultdict()

for j in resultados_Tiempos_Nivel_Jugador["erroresCodigo"]:
    res[j] = len(resultados_Tiempos_Nivel_Jugador["erroresCodigo"][j])

print(res)

defaultdict(None, {'pbgky': 10, 'auvqa': 11, 'gcpze': 13, 'uzzpo': 10, 'ftpnb': 11, 'eprdu': 9, 'zxvkj': 7, 'khadt': 9, 'loemd': 11, 'vsxld': 8, 'srjon': 10, 'qeqyj': 11, 'ikcar': 7, 'gleqe': 6, 'kmqou': 12, 'hjpbw': 9, 'ejeuw': 6, 'epqkm': 3, 'fkmtp': 11, 'zwzfv': 7, 'ipdcp': 16, 'xnpqf': 6, 'swvxx': 8, 'asopk': 13, 'oahey': 12, 'kkoif': 17, 'bhyhw': 14, 'cmbfi': 10, 'krahz': 11, 'pygoc': 11, 'dunul': 11, 'dwqwi': 13, 'rdiaf': 14, 'zcibf': 8, 'yvpmf': 10, 'grkij': 9, 'onyep': 12, 'yfohf': 7, 'accul': 7, 'edxmq': 9, 'tekwp': 10, 'tjfzy': 5, 'bdwyj': 8, 'sxipu': 6, 'ubmtx': 9, 'sjpkc': 10, 'nmveq': 4, 'camhi': 2, 'stiry': 4, 'mymho': 3, 'psido': 13, 'wjaxi': 17, 'fivin': 4, 'gguuc': 9, 'ewqbs': 11, 'hqmhq': 5, 'mjiso': 15, 'hpdxj': 3, 'lwryl': 10, 'ucdvy': 10, 'kkliq': 8, 'dvmlu': 4, 'yxewz': 5, 'gdljy': 5, 'tgwmn': 7, 'rziia': 4, 'zbgwk': 4, 'fyyya': 5, 'tinco': 3, 'ztyed': 1, 'yxjji': 1, 'mgaak': 13, 'qfsdj': 14, 'atlzs': 7, 'ngavz': 22, 'yazod': 8, 'tqgyp': 16, 'zdnso': 13, 'idmzi': 

In [13]:
filename = 'results-survey976727_pre.csv'
data = pd.read_csv(filename, sep=';', header=0)

data.drop(['submitdate', 'G00Q01[SQ01]', 'G02Q01', 'G02Q02', 'G02Q03', 'G02Q04', 'G03Q02[other]', 'id'], axis = 'columns', inplace=True)

def mezcla_edad_genero(edad, genero):
    return genero + ' ' + str(edad)

data['edad_genero'] = data.apply(lambda x: mezcla_edad_genero(x['G03Q01'], x['G03Q02']), axis=1)
#data

In [14]:
# Crear un DataFrame con los datos de los intentos medios de cada jugador

intentosJugadores = pd.DataFrame(intentosMedios_Individual["intentosIndividual"])
#intentosJugadores

# Convertir columnas en filas
df_stacked = intentosJugadores.stack().reset_index()
df_stacked.columns = ['nivel', 'token', 'intentos']

# Calcular promedio de intentos por jugador
promedio_intentos = df_stacked.groupby('token')['intentos'].mean()

# Crear DataFrame final
df_final = pd.DataFrame({'token': promedio_intentos.index, 'intentos': promedio_intentos.values})


# Convertir índice de df_tiempoInteraccion_Interaccion_Jugador en columna
df_tiempoInteraccion_Interaccion_Jugador2 = df_tiempoInteraccion_Interaccion_Jugador.stack().reset_index()
df_tiempoInteraccion_Interaccion_Jugador2.columns = ['nivel', 'token', 'tiempo']

# Calcular promedio de intentos por jugador
promedio_interaccion = df_tiempoInteraccion_Interaccion_Jugador2.groupby('token')['tiempo'].mean()

# Crear DataFrame final
df_final2 = pd.DataFrame({'token': promedio_interaccion.index, 'tiempo': promedio_interaccion.values})

# Unir con intentosJugadores usando merge en'token'
df_jugadores = pd.merge(df_final, df_final2, on=['token'])

# Unir con data usando merge en 'token'
df_global = pd.merge(df_jugadores, data, on='token')
#df_global
#df_jugadores


In [15]:
# Calcular los límites de los valores atípicos
# regla empírica, que establece que los valores atípicos se encuentran a más de 3 desviaciones estándar de la media.

Q1_edad = df_global['G03Q01'].quantile(0.25)
Q3_edad = df_global['G03Q01'].quantile(0.75)
IQR_edad = Q3_edad - Q1_edad
outlier_threshold_edad = Q3_edad + 1.5*IQR_edad

Q1_intentos = df_global['intentos'].quantile(0.25)
Q3_intentos = df_global['intentos'].quantile(0.75)
IQR_intentos = Q3_intentos - Q1_intentos
outlier_threshold_intentos = Q3_intentos + 1.5*IQR_intentos

df = df_global[(df_global['G03Q01'] <= outlier_threshold_edad) & (df_global['intentos'] <= outlier_threshold_intentos) & (df_global['tiempo'] <= outlier_threshold_intentos)]

# Crear una nueva columna que represente la mezcla al 50% de intentos y tiempo
df['mezcla_intentos_tiempo'] = (df['intentos'] + df['tiempo']) / 2

#df['mezcla_intentos_tiempo'] = df['intentos']*0.3 + df['tiempo']*0.7

#df['mezcla_intentos_tiempo'] = df['intentos']*0.7 + df['tiempo']*0.3


df


C:\Users\david\AppData\Local\Temp\ipykernel_24608\3069241159.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mezcla_intentos_tiempo'] = (df['intentos'] + df['tiempo']) / 2


,token,intentos,tiempo,G03Q01,G03Q02,edad_genero,mezcla_intentos_tiempo
3,asopk,1.885714,5.666881,14,Masculino,Masculino 14,3.776297
4,atlzs,2.423077,5.272170,14,Femenino,Femenino 14,3.847624
5,auvqa,2.043478,6.953159,14,Femenino,Femenino 14,4.498319
7,bdwyj,2.950000,6.006920,13,Masculino,Masculino 13,4.478460
8,bhyhw,1.892857,5.918490,14,Masculino,Masculino 14,3.905674
9,bjvnt,2.700000,3.000000,15,Femenino,Femenino 15,2.850000
11,buvgs,2.636364,6.534245,12,Femenino,Femenino 12,4.585304
13,cmbfi,1.516129,5.883715,13,Masculino,Masculino 13,3.699922
16,dunul,1.586207,6.514073,14,Otro,Otro 14,4.050140
19,eayrh,2.739130,5.227795,13,Femenino,Femenino 13,3.983462


In [16]:
# Crear gráfica 2D
fig = px.scatter(df, x='edad_genero', y='mezcla_intentos_tiempo', color='intentos', size='intentos',
                 opacity=0.7, color_continuous_scale='viridis')

# Configurar layout
fig.update_layout(xaxis_title='Edad_Genero', yaxis_title='Mezcla de Intentos y Tiempo', width=800, height=600)

# Mostrar gráfica
fig.show()




# con KMeans:

In [17]:
from sklearn.cluster import KMeans

# Crear un diccionario que asigne una etiqueta numérica única a cada combinación única de edad y género en la columna 'edad_genero'
etiquetas_edad_genero = {valor: indice for indice, valor in enumerate(df['edad_genero'].unique())}

# Convertir los valores de la columna 'edad_genero' en valores numéricos utilizando el diccionario de etiquetas
df['edad_genero_num'] = df['edad_genero'].apply(lambda x: etiquetas_edad_genero[x])

#Coger las columnas necesarias para crear la matriz de características
matriz_caracteristicas = df[['edad_genero_num', 'mezcla_intentos_tiempo']].values

# Normalizar los datos
matriz_caracteristicas_norm = StandardScaler().fit_transform(matriz_caracteristicas)

# Definimos o número de iteracións
n_iter = 20

# Obtemos os mesmos centroides iniciais que antes
SEED_VALUE = 190463  # NON CAMBIES ESTE SEED, así todos teremos os mesmos resultados
np.random.seed(SEED_VALUE)
centr_iniciais = matriz_caracteristicas_norm[np.random.choice(matriz_caracteristicas_norm.shape[0], size=5, replace=False)]

# Definimos o modelo
model = KMeans(n_clusters=len(centr_iniciais), init=centr_iniciais, n_init=1,
            max_iter=n_iter, algorithm='full', random_state=SEED_VALUE)

# Axustamos o modelo aos datos
np.random.seed(SEED_VALUE)
#df
agrupamento = model.fit(matriz_caracteristicas_norm)

# Crear la figura
fig = go.Figure(data=[go.Scatter(x=df['edad_genero'], y=df['mezcla_intentos_tiempo'], mode='markers',
                                 marker=dict(size=10, color=agrupamento.labels_ + 1, opacity=0.8,
                                             colorscale='Viridis'))])

# Configurar layout
fig.update_layout(title='Clustering de intentos',
                  xaxis_title='Edad_Genero',
                  yaxis_title='Mezcla de Intentos y Tiempo',
                  width=800, height=600)

# Mostrar gráfica
fig.show()


C:\Users\david\AppData\Local\Temp\ipykernel_24608\128020466.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\david\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1366: FutureWarning:

algorithm='full' is deprecated, it will be removed in 1.3. Using 'lloyd' instead.



# con DBSCAN:

In [18]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

# Convertir los valores de la columna 'edad_genero' en valores numéricos utilizando el diccionario de etiquetas
df['edad_genero_num'] = df['edad_genero'].apply(lambda x: etiquetas_edad_genero[x])

#Coger las columnas necesarias para crear la matriz de características
matriz_caracteristicas = df[['edad_genero_num', 'mezcla_intentos_tiempo']].values

# Normalizar los datos
#matriz_caracteristicas_norm = StandardScaler().fit_transform(matriz_caracteristicas)

# Definir el modelo
model = DBSCAN(eps=0.7, min_samples=5)

# Ajustar el modelo a los datos
agrupamento = model.fit(matriz_caracteristicas)

# Crear la figura
fig = go.Figure(data=[go.Scatter(x=df['edad_genero'], y=df['mezcla_intentos_tiempo'], mode='markers',
                                 marker=dict(size=10, color=agrupamento.labels_ + 1, opacity=0.8,
                                             colorscale='Viridis'))])

# Configurar layout
fig.update_layout(title='Clustering de intentos',
                  xaxis_title='Edad_Genero',
                  yaxis_title='Mezcla de Intentos y Tiempo',
                  width=800, height=600)

# Mostrar gráfica
fig.show()


C:\Users\david\AppData\Local\Temp\ipykernel_24608\1425290862.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# Con Hierarchical Agglomerative Clustering (HAC):

In [19]:
import numpy as np
import plotly.graph_objects as go
from scipy.cluster.hierarchy import linkage, fcluster
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

#Tecnicas de principal component analysis
#conjunto minimo de datos que explican una cirscuntancia
#Claro, el Principal Component Analysis (PCA) es una técnica de reducción de dimensionalidad que se utiliza en el análisis de datos y en el aprendizaje automático. El objetivo principal de PCA es transformar un conjunto de variables correlacionadas en un conjunto de variables no correlacionadas llamadas componentes principales.

#PCA es útil cuando se trabaja con un conjunto de datos con muchas variables, y se desea reducir la cantidad de variables manteniendo la mayor cantidad de información posible. Esto se logra al encontrar las direcciones en las que los datos tienen la mayor varianza y proyectar los datos en estas direcciones.

#El proceso de PCA se puede resumir en los siguientes pasos:

#Estandarizar los datos: Esto implica escalar las variables para que tengan una media de 0 y una desviación estándar de 1.
#Calcular la matriz de covarianza: La matriz de covarianza es una matriz cuadrada que contiene las covarianzas entre cada par de variables.
#Calcular los eigenvectores y eigenvalores de la matriz de covarianza: Los eigenvectores representan las direcciones en las que los datos tienen la mayor varianza, y los eigenvalores representan la magnitud de la varianza en cada dirección.
#Ordenar los eigenvectores según sus eigenvalores correspondientes en orden descendente.
#Seleccionar los primeros k eigenvectores, donde k es el número de componentes principales que se desea mantener.
#Proyectar los datos en el espacio de los k eigenvectores seleccionados.
#El resultado es un conjunto de datos transformado con menos dimensiones, que conserva la mayor cantidad de información posible del conjunto de datos original.


#elegir 2 sistemas cada uno, 1 mas, resumen. tracksrls frances si. uno de codigo abierto
#ver funcionalidades y visualizacion y analisis

#predecir si al jugador le va a ir bien o no segun los datos 

# Convertir los valores de la columna 'edad_genero' en valores numéricos utilizando el diccionario de etiquetas
df['edad_genero_num'] = df['edad_genero'].apply(lambda x: etiquetas_edad_genero[x])

#Coger las columnas necesarias para crear la matriz de características
matriz_caracteristicas = df[['edad_genero_num', 'mezcla_intentos_tiempo']].values

# Normalizar los datos
#matriz_caracteristicas_norm = StandardScaler().fit_transform(matriz_caracteristicas)

# Calcular la matriz de enlaces
Z = linkage(matriz_caracteristicas, method='ward')

# Calcular los valores de silueta para diferentes umbrales
scores = []
for i in range(2, len(Z)):
    clusters = fcluster(Z, t=i, criterion='maxclust')
    score = silhouette_score(matriz_caracteristicas, clusters)
    scores.append(score)

# Encontrar el umbral que maximiza el valor de silueta
umbral = np.argmax(scores) + 2

# Asignar los clusters en base al umbral
agrupamiento = fcluster(Z, t=umbral, criterion='maxclust')

# Crear la figura
fig = go.Figure(data=[go.Scatter(x=df['edad_genero'], y=df['mezcla_intentos_tiempo'], mode='markers',
                                 marker=dict(size=10, color=agrupamento.labels_ + 1, opacity=0.8,
                                             colorscale='Viridis'))])

# Configurar layout
fig.update_layout(title='Clustering de intentos',
                  xaxis_title='Edad_Genero',
                  yaxis_title='Mezcla de Intentos y Tiempo',
                  width=800, height=600)

# Mostrar gráfica
fig.show()


C:\Users\david\AppData\Local\Temp\ipykernel_24608\3746421276.py:30: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# Con Spectral Clustering

In [20]:
from sklearn.cluster import SpectralClustering

# Convertir los valores de la columna 'genero' en valores numéricos utilizando el diccionario de etiquetas
df['edad_genero_num'] = df['edad_genero'].apply(lambda x: etiquetas_edad_genero[x])

# Eliminar columnas innecesarias antes de crear la matriz de características
matriz_caracteristicas = df.drop(['G03Q02', 'token', 'G03Q01', 'edad_genero'], axis=1).values

# Normalizar los datos
matriz_caracteristicas_norm = StandardScaler().fit_transform(matriz_caracteristicas)

# Definimos el modelo
model = SpectralClustering(n_components=None, random_state=None)

# Axustamos el modelo aos datos
agrupamiento = model.fit_predict(matriz_caracteristicas_norm)

# Crear la figura
fig = go.Figure(data=[go.Scatter(x=df['edad_genero'], y=df['mezcla_intentos_tiempo'], mode='markers',
                                 marker=dict(size=10, color=agrupamento.labels_ + 1, opacity=0.8,
                                             colorscale='Viridis'))])

# Configurar layout
fig.update_layout(title='Clustering de intentos',
                  xaxis_title='Edad_Genero',
                  yaxis_title='Mezcla de Intentos y Tiempo',
                  width=800, height=600)

# Mostrar gráfica
fig.show()


C:\Users\david\AppData\Local\Temp\ipykernel_24608\3723826965.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [21]:
df['cluster'] = agrupamento.labels_
df

C:\Users\david\AppData\Local\Temp\ipykernel_24608\2127888770.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,token,intentos,tiempo,G03Q01,G03Q02,edad_genero,mezcla_intentos_tiempo,edad_genero_num,cluster
3,asopk,1.885714,5.666881,14,Masculino,Masculino 14,3.776297,0,0
4,atlzs,2.423077,5.272170,14,Femenino,Femenino 14,3.847624,1,1
5,auvqa,2.043478,6.953159,14,Femenino,Femenino 14,4.498319,1,1
7,bdwyj,2.950000,6.006920,13,Masculino,Masculino 13,4.478460,2,2
8,bhyhw,1.892857,5.918490,14,Masculino,Masculino 14,3.905674,0,0
9,bjvnt,2.700000,3.000000,15,Femenino,Femenino 15,2.850000,3,-1
11,buvgs,2.636364,6.534245,12,Femenino,Femenino 12,4.585304,4,3
13,cmbfi,1.516129,5.883715,13,Masculino,Masculino 13,3.699922,2,2
16,dunul,1.586207,6.514073,14,Otro,Otro 14,4.050140,5,-1
19,eayrh,2.739130,5.227795,13,Femenino,Femenino 13,3.983462,6,4


# Anteriores

In [22]:


# Pivotear el DataFrame para tener un formato largo
intentos_df_long = pd.melt(intentosJugadores.reset_index(), id_vars=['index'], var_name='Jugador', value_name='Intentos')
intentos_df_long.columns = ['Nivel', 'Jugador', 'Intentos']

# Agrupar los intentos de cada jugador por nivel y tomar la media
intentos_jugador_nivel = intentos_df_long.groupby(['Jugador', 'Nivel'])['Intentos'].mean().unstack()

# Unir la tabla de intentos de cada jugador con los datos originales
df = data_filtrado.merge(intentos_jugador_nivel, left_on='token', right_index=True)

# Seleccionar las variables a utilizar para el clustering
X = df.iloc[:, 3:] # Seleccionar las columnas correspondientes a los niveles

# Imputar los valores faltantes
imputer = SimpleImputer(strategy='median')
X_imputed = imputer.fit_transform(X)

# Normalizar las variables utilizando StandardScaler
scaler = StandardScaler()
X_norm = scaler.fit_transform(X_imputed)

# Definir el número de clusters y entrenar el modelo de K-means
num_clusters = 4
kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(X_norm)

# Asignar a cada jugador el cluster correspondiente
clusters = kmeans.labels_
df['Cluster'] = clusters

# Calcular la media de los números de intentos en cada cluster
means = df.groupby(['G03Q02', 'G03Q01', 'Cluster'])[X.columns].mean()

# Mostrar los resultados
print(means) #Dar a abrir en un editor de texto para ver mejor la tabla
#intentosJugadores


NameError: name 'data_filtrado' is not defined

In [ ]:
tiemposJugadores = pd.DataFrame(tiemposMedios["tiemposIndividuales"])

# Aplicar la función int() solamente a los valores que no son NaN en el DataFrame
tiemposJugadores = tiemposJugadores.applymap(lambda x: int(x) if x == x else x)

# Pivotear el DataFrame para tener un formato largo
tiempos_df_long = pd.melt(tiemposJugadores.reset_index(), id_vars=['index'], var_name='Jugador', value_name='Intentos')
tiempos_df_long.columns = ['Nivel', 'Jugador', 'Intentos']

# Agrupar los intentos de cada jugador por nivel y tomar la media
tiempos_jugador_nivel = tiempos_df_long.groupby(['Jugador', 'Nivel'])['Intentos'].mean().unstack()

# Unir la tabla de intentos de cada jugador con los datos originales
df = data_filtrado.merge(tiempos_jugador_nivel, left_on='token', right_index=True)

# Seleccionar las variables a utilizar para el clustering
X = df.iloc[:, 3:] # Seleccionar las columnas correspondientes a los niveles

# Imputar los valores faltantes
imputer = SimpleImputer(strategy='median')
X_imputed = imputer.fit_transform(X)

# Normalizar las variables utilizando StandardScaler
scaler = StandardScaler()
X_norm = scaler.fit_transform(X_imputed)

# Definir el número de clusters y entrenar el modelo de K-means
num_clusters = 4
kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(X_norm)

# Asignar a cada jugador el cluster correspondiente
clusters = kmeans.labels_
df['Cluster'] = clusters

# Calcular la media de los números de intentos en cada cluster
means = df.groupby(['G03Q02', 'G03Q01', 'Cluster'])[X.columns].mean()

# Mostrar los resultados
print(means)
# tiemposJugadores


In [ ]:
media_intentos = df.groupby(['G03Q02', 'G03Q01', 'Cluster'])[X.columns].mean().reset_index()
media_intentos = pd.melt(media_intentos, id_vars=['G03Q02', 'G03Q01', 'Cluster'], var_name='Nivel', value_name='Intentos')

import seaborn as sns
import matplotlib.pyplot as plt

# Crear una figura para cada nivel
for nivel in intentos_df_long['Nivel'].unique():
    # Seleccionar los datos del nivel y generar una tabla pivote con los valores medios de intentos por género
    df_nivel = media_intentos[media_intentos['Nivel'] == nivel]
    pivot_table = pd.pivot_table(df_nivel, values='Intentos', index=['G03Q01'], columns=['Cluster'], aggfunc=np.mean)
    
    # Crear un gráfico de barras apiladas para comparar los valores medios de intentos por género en cada cluster
    sns.catplot(x='G03Q01', y='Intentos', hue='Cluster', data=df_nivel, kind='bar', ci=None, legend_out=False)
    plt.title(f'Nivel {nivel}')
    plt.show()
